# Prediction Models Using Decomposed Components

This notebook demonstrates how to build prediction models using the decomposed components from Z1 time series data.

## 1. Setup and Imports

In [ ]:
# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# ML imports
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

# Project imports
import sys
sys.path.append('..')  # Add parent directory to path

from src.analysis.feature_engineering import FeatureEngineer
from src.models.prediction_models import TimeSeriesPredictor

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

## 2. Load Decomposed Components

In [ ]:
# Load decomposed components from previous notebook
data_dir = Path('../data/decomposed')

components = {}
for comp_file in data_dir.glob('*_components.parquet'):
    comp_name = comp_file.stem.replace('_components', '')
    components[comp_name] = pd.read_parquet(comp_file)
    print(f"Loaded {comp_name}: {components[comp_name].shape}")

# Load metadata
with open(data_dir / 'decomposition_metadata.json', 'r') as f:
    metadata = json.load(f)

print(f"\nTotal series: {len(metadata['series'])}")
print(f"Components: {metadata['components']}")

## 3. Feature Engineering

In [ ]:
# Initialize feature engineer
feature_engineer = FeatureEngineer()

# Select target variable for prediction
target_series = metadata['series'][0]  # First series as example
print(f"Target series for prediction: {target_series}")

# Extract cycle component as primary feature
if 'cycle' in components:
    cycle_df = components['cycle'].iloc[30:]  # Skip initial observations
    print(f"\nCycle component shape: {cycle_df.shape}")
else:
    print("No cycle component found. Using trend instead.")
    cycle_df = components['trend'].iloc[30:]

# Create lagged features
max_lags = 8
lagged_features = []

for lag in range(1, max_lags + 1):
    lagged_df = cycle_df.shift(lag).add_suffix(f'_lag{lag}')
    lagged_features.append(lagged_df)

# Combine all features
feature_df = pd.concat([cycle_df] + lagged_features, axis=1)
feature_df = feature_df.dropna()

print(f"\nTotal features created: {feature_df.shape[1]}")
print(f"Feature matrix shape: {feature_df.shape}")

## 4. Prepare Target Variable

In [ ]:
# Use the target series cycle component as the target
if target_series in cycle_df.columns:
    y = cycle_df[target_series].loc[feature_df.index]
else:
    # If target not in cycle, use the first available series
    target_series = cycle_df.columns[0]
    y = cycle_df[target_series].loc[feature_df.index]
    print(f"Updated target series: {target_series}")

# Remove target from features
X = feature_df.drop(columns=[col for col in feature_df.columns if target_series in col])

print(f"\nFeatures shape: {X.shape}")
print(f"Target shape: {y.shape}")

# Visualize target variable
plt.figure(figsize=(12, 4))
plt.plot(y.index, y.values)
plt.title(f'Target Variable: {target_series} (Cycle Component)')
plt.xlabel('Date')
plt.ylabel('Value')
plt.grid(True, alpha=0.3)
plt.show()

## 5. Train-Test Split

In [ ]:
# Time series split - maintain temporal order
split_ratio = 0.8
split_point = int(len(X) * split_ratio)

X_train, X_test = X.iloc[:split_point], X.iloc[split_point:]
y_train, y_test = y.iloc[:split_point], y.iloc[split_point:]

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print(f"\nTraining period: {X_train.index.min()} to {X_train.index.max()}")
print(f"Test period: {X_test.index.min()} to {X_test.index.max()}")

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## 6. LASSO Regression with Feature Selection

In [ ]:
# Train LASSO model
lasso = Lasso(alpha=0.003, max_iter=10000)
lasso.fit(X_train_scaled, y_train)

# Make predictions
y_pred_lasso = lasso.predict(X_test_scaled)

# Evaluate performance
mse_lasso = mean_squared_error(y_test, y_pred_lasso)
mae_lasso = mean_absolute_error(y_test, y_pred_lasso)
r2_lasso = r2_score(y_test, y_pred_lasso)

print("LASSO Model Performance:")
print("=" * 50)
print(f"MSE: {mse_lasso:.6f}")
print(f"MAE: {mae_lasso:.6f}")
print(f"R²: {r2_lasso:.4f}")

# Feature importance from LASSO coefficients
feature_importance = pd.DataFrame({
    'Feature': X_train.columns,
    'Coefficient': lasso.coef_
})
feature_importance['Abs_Coefficient'] = np.abs(feature_importance['Coefficient'])
feature_importance = feature_importance.sort_values('Abs_Coefficient', ascending=False)

# Display top features
print("\nTop 20 Important Features:")
print(feature_importance.head(20)[['Feature', 'Coefficient']])

In [ ]:
# Visualize LASSO predictions
plt.figure(figsize=(12, 6))
plt.plot(y_test.index, y_test.values, 'b-', label='Actual', linewidth=2)
plt.plot(y_test.index, y_pred_lasso, 'r--', label='LASSO Predicted', linewidth=2)
plt.title(f'LASSO Predictions vs Actual - {target_series}')
plt.xlabel('Date')
plt.ylabel('Value')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Residual plot
residuals_lasso = y_test - y_pred_lasso
plt.figure(figsize=(12, 4))
plt.plot(y_test.index, residuals_lasso, 'g-', alpha=0.7)
plt.axhline(y=0, color='black', linestyle='--')
plt.title('LASSO Prediction Residuals')
plt.xlabel('Date')
plt.ylabel('Residual')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 7. XGBoost Model

In [ ]:
# Configure XGBoost parameters
xgb_params = {
    'lambda': 0.05,
    'alpha': 0.02,
    'gamma': 0.000001,
    'colsample_bytree': 0.43,
    'subsample': 0.99,
    'learning_rate': 0.05,
    'n_estimators': 300,
    'max_depth': 4,
    'min_child_weight': 3,
    'objective': 'reg:squarederror',
    'random_state': 42
}

# Train XGBoost model
xgb_model = xgb.XGBRegressor(**xgb_params)
xgb_model.fit(X_train, y_train, 
              eval_set=[(X_test, y_test)],
              early_stopping_rounds=50,
              verbose=False)

# Make predictions
y_pred_xgb = xgb_model.predict(X_test)

# Evaluate performance
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)

print("XGBoost Model Performance:")
print("=" * 50)
print(f"MSE: {mse_xgb:.6f}")
print(f"MAE: {mae_xgb:.6f}")
print(f"R²: {r2_xgb:.4f}")
print(f"\nBest iteration: {xgb_model.best_iteration}")

In [ ]:
# Feature importance from XGBoost
xgb_importance = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': xgb_model.feature_importances_
}).sort_values('Importance', ascending=False)

# Plot feature importance
plt.figure(figsize=(10, 8))
top_features = xgb_importance.head(20)
plt.barh(range(len(top_features)), top_features['Importance'])
plt.yticks(range(len(top_features)), top_features['Feature'])
plt.xlabel('Feature Importance')
plt.title('Top 20 Features - XGBoost')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## 8. Random Forest Model

In [ ]:
# Train Random Forest model
rf_model = RandomForestRegressor(
    n_estimators=500,
    max_depth=5,
    min_samples_split=10,
    min_samples_leaf=5,
    random_state=42,
    n_jobs=-1
)

rf_model.fit(X_train, y_train)

# Make predictions
y_pred_rf = rf_model.predict(X_test)

# Evaluate performance
mse_rf = mean_squared_error(y_test, y_pred_rf)
mae_rf = mean_absolute_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print("Random Forest Model Performance:")
print("=" * 50)
print(f"MSE: {mse_rf:.6f}")
print(f"MAE: {mae_rf:.6f}")
print(f"R²: {r2_rf:.4f}")

## 9. Model Comparison

In [ ]:
# Compare all models
model_comparison = pd.DataFrame({
    'Model': ['LASSO', 'XGBoost', 'Random Forest'],
    'MSE': [mse_lasso, mse_xgb, mse_rf],
    'MAE': [mae_lasso, mae_xgb, mae_rf],
    'R²': [r2_lasso, r2_xgb, r2_rf]
})

print("Model Performance Comparison:")
print("=" * 50)
print(model_comparison.round(4))

# Visualize model predictions
fig, axes = plt.subplots(3, 1, figsize=(12, 12))

models = [
    ('LASSO', y_pred_lasso, 'red'),
    ('XGBoost', y_pred_xgb, 'green'),
    ('Random Forest', y_pred_rf, 'orange')
]

for idx, (name, pred, color) in enumerate(models):
    axes[idx].plot(y_test.index, y_test.values, 'b-', label='Actual', linewidth=2, alpha=0.7)
    axes[idx].plot(y_test.index, pred, color=color, linestyle='--', label=f'{name} Predicted', linewidth=2)
    axes[idx].set_title(f'{name} Model Predictions')
    axes[idx].set_ylabel('Value')
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)

axes[-1].set_xlabel('Date')
plt.tight_layout()
plt.show()

## 10. Ensemble Prediction

In [ ]:
# Create ensemble prediction (weighted average)
# Weight by R² scores
weights = np.array([r2_lasso, r2_xgb, r2_rf])
weights = weights / weights.sum()

y_pred_ensemble = (
    weights[0] * y_pred_lasso +
    weights[1] * y_pred_xgb +
    weights[2] * y_pred_rf
)

# Evaluate ensemble
mse_ensemble = mean_squared_error(y_test, y_pred_ensemble)
mae_ensemble = mean_absolute_error(y_test, y_pred_ensemble)
r2_ensemble = r2_score(y_test, y_pred_ensemble)

print("Ensemble Model Performance:")
print("=" * 50)
print(f"Weights: LASSO={weights[0]:.3f}, XGBoost={weights[1]:.3f}, RF={weights[2]:.3f}")
print(f"MSE: {mse_ensemble:.6f}")
print(f"MAE: {mae_ensemble:.6f}")
print(f"R²: {r2_ensemble:.4f}")

# Plot ensemble predictions
plt.figure(figsize=(12, 6))
plt.plot(y_test.index, y_test.values, 'b-', label='Actual', linewidth=2)
plt.plot(y_test.index, y_pred_ensemble, 'purple', linestyle='--', label='Ensemble Predicted', linewidth=2)
plt.title(f'Ensemble Model Predictions - {target_series}')
plt.xlabel('Date')
plt.ylabel('Value')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 11. Time Series Cross-Validation

In [ ]:
# Perform time series cross-validation
tscv = TimeSeriesSplit(n_splits=5)
cv_scores = {'LASSO': [], 'XGBoost': [], 'Random Forest': []}

print("Time Series Cross-Validation:")
print("=" * 50)

for fold, (train_idx, val_idx) in enumerate(tscv.split(X)):
    X_cv_train, X_cv_val = X.iloc[train_idx], X.iloc[val_idx]
    y_cv_train, y_cv_val = y.iloc[train_idx], y.iloc[val_idx]
    
    # Scale data
    scaler_cv = StandardScaler()
    X_cv_train_scaled = scaler_cv.fit_transform(X_cv_train)
    X_cv_val_scaled = scaler_cv.transform(X_cv_val)
    
    # LASSO
    lasso_cv = Lasso(alpha=0.003, max_iter=10000)
    lasso_cv.fit(X_cv_train_scaled, y_cv_train)
    cv_scores['LASSO'].append(r2_score(y_cv_val, lasso_cv.predict(X_cv_val_scaled)))
    
    # XGBoost
    xgb_cv = xgb.XGBRegressor(**xgb_params)
    xgb_cv.fit(X_cv_train, y_cv_train, verbose=False)
    cv_scores['XGBoost'].append(r2_score(y_cv_val, xgb_cv.predict(X_cv_val)))
    
    # Random Forest
    rf_cv = RandomForestRegressor(n_estimators=500, max_depth=5, random_state=42, n_jobs=-1)
    rf_cv.fit(X_cv_train, y_cv_train)
    cv_scores['Random Forest'].append(r2_score(y_cv_val, rf_cv.predict(X_cv_val)))
    
    print(f"Fold {fold+1}: LASSO={cv_scores['LASSO'][-1]:.3f}, "
          f"XGBoost={cv_scores['XGBoost'][-1]:.3f}, "
          f"RF={cv_scores['Random Forest'][-1]:.3f}")

# Summary statistics
print("\nCross-Validation Summary (R² scores):")
for model, scores in cv_scores.items():
    print(f"{model}: Mean={np.mean(scores):.3f}, Std={np.std(scores):.3f}")

## 12. Feature Analysis and Interpretation

In [ ]:
# Analyze most important features across models
# Get top features from each model
top_n = 10

# LASSO top features
lasso_top = feature_importance.head(top_n)['Feature'].tolist()

# XGBoost top features  
xgb_top = xgb_importance.head(top_n)['Feature'].tolist()

# Random Forest top features
rf_importance = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': rf_model.feature_importances_
}).sort_values('Importance', ascending=False)
rf_top = rf_importance.head(top_n)['Feature'].tolist()

# Find common important features
common_features = set(lasso_top) & set(xgb_top) & set(rf_top)
print(f"Common top features across all models: {len(common_features)}")
for feat in common_features:
    print(f"  - {feat}")

# Create feature importance comparison
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# LASSO
axes[0].barh(range(top_n), feature_importance.head(top_n)['Abs_Coefficient'])
axes[0].set_yticks(range(top_n))
axes[0].set_yticklabels(feature_importance.head(top_n)['Feature'])
axes[0].set_title('LASSO - Top Features')
axes[0].invert_yaxis()

# XGBoost
axes[1].barh(range(top_n), xgb_importance.head(top_n)['Importance'])
axes[1].set_yticks(range(top_n))
axes[1].set_yticklabels(xgb_importance.head(top_n)['Feature'])
axes[1].set_title('XGBoost - Top Features')
axes[1].invert_yaxis()

# Random Forest
axes[2].barh(range(top_n), rf_importance.head(top_n)['Importance'])
axes[2].set_yticks(range(top_n))
axes[2].set_yticklabels(rf_importance.head(top_n)['Feature'])
axes[2].set_title('Random Forest - Top Features')
axes[2].invert_yaxis()

plt.tight_layout()
plt.show()

## 13. Out-of-Sample Forecasting

In [ ]:
# Generate out-of-sample forecasts
# Use the best performing model (based on R²)
best_model_name = model_comparison.loc[model_comparison['R²'].idxmax(), 'Model']
print(f"Best model for forecasting: {best_model_name}")

# Retrain on full dataset
X_full = pd.concat([X_train, X_test])
y_full = pd.concat([y_train, y_test])

if best_model_name == 'XGBoost':
    final_model = xgb.XGBRegressor(**xgb_params)
    final_model.fit(X_full, y_full, verbose=False)
elif best_model_name == 'Random Forest':
    final_model = RandomForestRegressor(n_estimators=500, max_depth=5, random_state=42, n_jobs=-1)
    final_model.fit(X_full, y_full)
else:  # LASSO
    X_full_scaled = scaler.fit_transform(X_full)
    final_model = Lasso(alpha=0.003, max_iter=10000)
    final_model.fit(X_full_scaled, y_full)

print("\nModel retrained on full dataset for forecasting")

# Note: For actual forecasting, you would need to:
# 1. Generate future feature values (using lagged predictions)
# 2. Make recursive predictions
# 3. Transform predictions back to original scale

## 14. Save Models and Results

In [ ]:
# Save trained models and results
import joblib

output_dir = Path('../models')
output_dir.mkdir(exist_ok=True)

# Save models
joblib.dump(lasso, output_dir / 'lasso_model.pkl')
joblib.dump(xgb_model, output_dir / 'xgboost_model.pkl')
joblib.dump(rf_model, output_dir / 'random_forest_model.pkl')
joblib.dump(scaler, output_dir / 'feature_scaler.pkl')

# Save predictions
predictions_df = pd.DataFrame({
    'actual': y_test,
    'lasso': y_pred_lasso,
    'xgboost': y_pred_xgb,
    'random_forest': y_pred_rf,
    'ensemble': y_pred_ensemble
}, index=y_test.index)

predictions_df.to_parquet(output_dir / 'predictions.parquet')

# Save model performance
model_comparison.to_csv(output_dir / 'model_performance.csv', index=False)

# Save feature importance
feature_importance.to_csv(output_dir / 'feature_importance_lasso.csv', index=False)
xgb_importance.to_csv(output_dir / 'feature_importance_xgboost.csv', index=False)
rf_importance.to_csv(output_dir / 'feature_importance_rf.csv', index=False)

print("\n✓ Models and results saved successfully!")
print(f"\nSaved to: {output_dir}")